**Метод наименьших квадратов** - это математический метод, применяемый для решения различных задач, основанный на минимизации суммы квадратов отклонений некоторых функций от экспериментальных входных данных.
Если говорить простыми словами: пусть у нас есть параметры $x, y$. Нам бы хотелось найти линейную функцию $f(x_i)$, которая будет "максимально близко" давать значения к   $y_i$. Во первых, точность ("максимально близко") будет измерятся как сумма квадратов отклонений: $\sum_{i = 1}^n (f(x_i) - y_i)^2$. Именно её мы постараемся минимизировать.

Ниже представлен пример с датасетом про квартиры. Построим линейную функцию, которая от значения площади квартиры будет предсказывать цену.


In [2]:
#Для начала загрузим нужные библиотеки
import pandas as pd
import numpy as np

Выгружаем данные. Этот датасет с kaggle , в котором хранится информация по квартирам: цена, площать, кол-во комнат и т.п. Попробуем обучить модель и оценить, с какой точность она будет оценивать квартиры на основе некоторых параметров.

In [3]:
data = pd.read_csv('Housing.csv')
data.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [17]:
from scipy.optimize import minimize
 
#Зададим параметры: х - площадь, у - цена
x = np.array(data['area'])
y = np.array(data['price'])
 
# Возьмём сумму квадратов отклонений, как функцию ошибки 
def error_fun(coefficients):
    return np.sum((np.polyval(coefficients, x) - y) ** 2)
 
# Находим коэффициенты аппроксимирующей функции
result = minimize(error_fun, [0, 0])
 
# Получаем значения аппроксимирующей функции
approximation = np.polyval(result.x, x)
 

#Выводим получившуюся функцию
print(f'f(x) = {result.x[0]}x + {result.x[1]}')
# Получим: f(x) = 461.9840518609294x + 2394815.325258006



f(x) = 461.9840518609294x + 2394815.325258006


Итак, мы получили коэффиценты, но надо ещё посчитать ошибку: сумму квадратов отклонений

In [15]:
print('Ошибка:', error_fun(result.x))
#Получим: 1356460561589906.0

Ошибка: 1356460561589906.0


Но если кто-то не доверяет функции minimize, то можно все эти вычисления проделать самим. Для этого нам понадобиться 1 курс линейной алгебры и волшебная библиотека linalg :)

Если коротко, то коэффиценты искомой линейной функции - это некое псевдорешение системы 
$$
\begin{cases}
a_1 x_{11} + a_2 x_{12} + ... + a_n x_{1n} + a_{n + 1} = y_1 \\
a_1 x_{21} + a_2 x_{22} + ... + a_n x_{2n} + a_{n + 1} = y_2 \\
\dots \\
a_1 x_{n1} + a_2 x_{n2} + ... + a_n x_{nn} + a_{n + 1}= y_n
\end{cases}
$$
Или если всё записывать через матрицы, то $Xa = Y$, где $a$ - это столбец коэффицентов, который мы пытаемся найти, $X$ - матрицы из параметров со столбцом из 1, $y$ - стобец из результатов (правая сторона всех равенств). И существует несложная формула, с помощью которой можно найти $a$:
$$
a = (X^TX)^{-1}X^Ty
$$

Давайте посчитаем через неё, чтобы удостовериться, что функция $minimize$ не ошиблась

In [18]:
# Добавим столбец из 1 и создадим матрицу Х
X = np.concatenate(([np.array(data['area'])], [np.ones(len(data['area']))]), axis=0).T 
print

# Воспользуемся формулой
k, b = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(np.array(data['price']))

# Выводим результат
print(f"f(x) = {k} * x + {b}")
# Получим: f(x) = 461.97489427278356 * x + 2387308.4823964285

f(x) = 461.97489427278356 * x + 2387308.4823964285


Почти ничем не отличается, но на всякий случай, посчитаем ошибку

In [19]:
print('Ошибка:', error_fun([k, b]))
#Получим: 1356429462008458.2

Ошибка: 1356429462008458.2


И ошибки тоже совпали, значить код работает верно!

Заметим, что ошибка-неточность большая. Но тут играют роль играет несколько моментов:
* Тут идёт сумма **квадратов**, и число получается большим
* Данных немного, всего 545
* Логично, что цена квартиры зависит не только от площади, и тем более не зависит линейно 


Попробуем исправить этот момент и посчитаем функцию (тоже линейную), но которая зависит не только от площади, но также количества комнат, наличием горячей воды и обогревания, отделкой и тому подобное. 

Для начала изменим все колонки так, чтобы все данные в них были числовыми.

In [46]:
dt = data.copy()
dt['mainroad'] = np.where(data['mainroad'] == 'yes', 1, 0)
dt['guestroom'] = np.where(data['guestroom'] == 'yes', 1, 0)
dt['basement'] = np.where(data['basement'] == 'yes', 1, 0)
dt['hotwaterheating'] = np.where(data['hotwaterheating'] == 'yes', 1, 0)
dt['airconditioning'] = np.where(data['airconditioning'] == 'yes', 1, 0)
dt['prefarea'] = np.where(data['prefarea'] == 'yes', 1, 0)

a = dict(zip(data['furnishingstatus'].unique().tolist(), [2, 1, 0]))
dt['furnishingstatus'] =  data['furnishingstatus'].map(a)

all = dt.drop('price', axis=1)

[[7420    4    2 ...    2    1    2]
 [8960    4    4 ...    3    0    2]
 [9960    3    2 ...    2    1    1]
 ...
 [3620    2    1 ...    0    0    0]
 [2910    3    1 ...    0    0    2]
 [3850    3    1 ...    0    0    0]]


In [61]:
# Добавим столбец из 1 и создадим матрицу Х
X = np.concatenate((np.array(all).T, [np.ones(len(data['area']))]), axis=0).T

# Воспользуемся формулой
coef = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(np.array(data['price']))

# Выводим результат
print(coef)
# Получим вот такие коээфиценты: [ 2.43906887e+02  1.19474386e+05  9.88888287e+05  4.50391518e+05
#   4.23100741e+05  2.98030507e+05  3.57926360e+05  8.72936027e+05
#   8.53633595e+05  2.79785637e+05  6.47055599e+05  2.13187781e+05
#  -3.24509050e+05] 

[ 2.43906887e+02  1.19474386e+05  9.88888287e+05  4.50391518e+05
  4.23100741e+05  2.98030507e+05  3.57926360e+05  8.72936027e+05
  8.53633595e+05  2.79785637e+05  6.47055599e+05  2.13187781e+05
 -3.24509050e+05]


Получим большую функцию с 13 коэффицентами:

$$
f(x)= 243.9068872689059x_1 + 119474.38646740332x_2 + 988888.2873006214x_3 \\
+ 450391.5176025255x_4 + 423100.74115655164x_5 + 298030.50677958474x_6 + 357926.36003238487x_7 \\
+ 872936.0271410313x_8 + 853633.5948889686x_9 + 279785.63738587266x_10  \\
+ 647055.5987220603x_11 + 213187.78145772417x_12 - 324509.04986334254
$$

Но насколько изменилась точность предсказаний...



In [68]:
#Напишем немного другую функцию подсчёта
def squar_sum(pred, y):
    su = 0
    for i in range(len(y)):
        su += (y[i] - pred[i]) * (y[i] - pred[i])
    return su


pred = X @ np.asarray(coef).T
print('Ошибка:', squar_sum(pred, dt.price.tolist()))
# #Ошибка: 608895018186504.6

Ошибка: 608895018186504.6


Параметр - только площадь: 1356429462008458.2 \
Несколько параметров:      608895018186504.6

Стало меньше, что логично, так как теперь функция ориентируется на большее количество параметров!
\
\
Таким образом, мы продемонстрировали несложный пример работы МНК 

Теперь аналогичным образов найдём функцию, но уже с помощью **Метода Наименьших Модулей**. Этот метод очень похож на МНК, за исключенеим того, что вместо квадратов, в минимизации суммы используются модули. 

In [71]:
import numpy as np
from scipy.optimize import minimize
 
# Задаем набор данных
x = np.array(data['area'])
y = np.array(data['price'])
 
# Возьмём сумму модулей отклонений, как функцию ошибки 
def error_fun(coefficients):
    return np.sum(np.abs(np.polyval(coefficients, x) - y))

 
# Находим коэффициенты аппроксимирующей функции
result = minimize(error_fun, [0, 0])
 
# Получаем значения аппроксимирующей функции
approximation = np.polyval(result.x, x)

# Выводим получившуюся функцию
print(f'f(x) = {result.x[0]}x + {result.x[1]}')
# Получим: f(x) = 489.05895115498197x + 2050603.2126229063


f(x) = 489.05895115498197x + 2050603.2126229063


В принципе, отличие функции от предыдущей небольшое. Давайте посмотрим на ошибку

In [72]:
print("Ошибка:", error_fun(result.x))
# Получаем: 631824343.0218697

Ошибка: 631824343.0218697


Встаёт вопрос, какой метод лучше. Для начала давайте сверим ошибки. Единсвтенное, так как МНК предполагает возведение в квадрат, для более понятного сравнения, возьмём корень из получившейся ошибки:
$$
МНК = \sqrt{1356460561589906.0} = 36830158.31611244 \approx 36830158
$$
Теперь просто сравним с ошибкой метода модулей:
$$
МНК = 36830158 < 631824343 = МНМ
$$
Получается, что МНК рабоатет лучше. Но есть ещё одна причина, почему часто берут именно квадрат, а не модуль: любая степеная функция дифференцируема, следовательно, её проще минимизировать.